In [ ]:
ALIAS=my_sample_name
PE_1=/path/to/paired_end_read_1
PE_2=/path/to/paired_end_read_2
MP1_1=/path/to/mate_pair_1/read_1
MP1_2=/path/to/mate_pair_1/read_2
MP2_1=/path/to/mate_pair_2/read_1
MP2_2=/path/to/mate_pair_2/read_2

In [ ]:
TRIMMOMATIC_JAR=/path/to/trimmomatic-0.32.jar
java -jar $TRIMMOMATIC_JAR PE -threads $NSLOTS $PE_1 $PE_2 \
  ${ALIAS}_pe_R1.c.fq ${ALIAS}_pe_U1.c.fq \
  ${ALIAS}_pe_R2.c.fq ${ALIAS}_pe_U2.c.fq \
  ILLUMINACLIP:Adapters.fasta:2:30:10 LEADING:3 TRAILING:3 MAXINFO:220:0.1 MINLEN:70
java -jar $TRIMMOMATIC_JAR PE -threads $NSLOTS $MP1_1 $MP1_2 \
  ${ALIAS}_mp1_R1.c.fq ${ALIAS}_mp1_U1.c.fq \
  ${ALIAS}_mp1_R2.c.fq ${ALIAS}_mp1_U2.c.fq \
  ILLUMINACLIP:Adapters.fasta:2:30:10 LEADING:3 TRAILING:3 MAXINFO:220:0.1 MINLEN:70
java -jar $TRIMMOMATIC_JAR PE -threads $NSLOTS $MP2_1 $MP2_2 \
  ${ALIAS}_mp2_R1.c.fq ${ALIAS}_mp2_U1.c.fq \
  ${ALIAS}_mp2_R2.c.fq ${ALIAS}_mp2_U2.c.fq \
  ILLUMINACLIP:Adapters.fasta:2:30:10 LEADING:3 TRAILING:3 MAXINFO:220:0.1 MINLEN:70

#concatenate single reads
cat *U[12].c.fq > ${ALIAS}_se.i.fq


In [ ]:
for kind in pe mp1 mp2
do
  seqtk mergepe ${ALIAS}_${kind}_R1.c.fq ${ALIAS}_${kind}_R2.c.fq > ${ALIAS}_${kind}.i.fq
done  

In [ ]:
for kind in pe mp1 mp2
do
  median_filter ${ALIAS}_${kind}.i.fq > ${ALIAS}_${kind}.mf.fq
done

In [ ]:
for kind in pe mp1 mp2
do
  extract-paired-reads.py ${ALIAS}_${kind}.mf.fq
done



In [ ]:
#concatenate single reads
cat *.se ${ALIAS}_se.mf.fq | gzip > ${ALIAS}_se.fq.gz

#renaming paires reads, compress
for kind in pe mp1 mp2
do
  mv ${ALIAS}_${kind}.mf.fq.pe ${ALIAS}_${kind}.fq
  gzip ${ALIAS}_${kind}.fq
done

In [ ]:
spades.py -t $NSLOTS --only-error-correction -o spades_output \
  --pe1-12 ${ALIAS}_pe.fq.gz --pe1-s ${ALIAS}_se.fq.gz \
  --hqmp1-12 ${ALIAS}_mp1.fq.gz --hqmp2-12 ${ALIAS}_mp2.fq.gz

In [ ]:
cd spades_output/corrected

gunzip ${ALIAS}_*_*.00.*_0.cor.fastq.gz

${ALIAS}_pe_1.00.2_0.cor.fastq ${ALIAS}_pe_2.00.2_0.cor.fastq > ${ALIAS}_pe.ec.fq
${ALIAS}_mp1_1.00.0_0.cor.fastq ${ALIAS}_mp1_2.00.0_0.cor.fastq > ${ALIAS}_mp1.ec.fq
${ALIAS}_mp2_1.00.1_0.cor.fastq ${ALIAS}_mp2_2.00.1_0.cor.fastq > ${ALIAS}_mp2.ec.fq

mv *.ec.fq ../../
cd ../../

In [ ]:
R_SCRIPTS=/path/to/kmergenie/R_scripts
ls *.ec.fq > ecfiles.txt
specialk ecfiles.txt -t $NSLOTS -l 51 -k 51 -o ${ALIAS}or
cp $R_SCRIPTS/*.r .
ACTUAL_COV=$(Rscript get-peak.r ${ALIAS}or-k51.histo)

FRACTION=$(python -c "print $TARGET_COV/$ACTUAL_COV")
echo "COV=$TARGET_COV" > coverage.txt

seqtk sample -s $A_NUMBER_1 ${ALIAS}_pe.ec.fq $FRACTION > ${ALIAS}_pe.ec.keep.fq
seqtk sample -s $A_NUMBER_2 ${ALIAS}_mp1.ec.fq $FRACTION > ${ALIAS}_mp1.ec.keep.fq
seqtk sample -s $A_NUMBER_2 ${ALIAS}_mp2.ec.fq $FRACTION > ${ALIAS}_mp2.ec.keep.fq
seqtk sample -s $A_NUMBER_1 ${ALIAS}_se.ec.fq $FRACTION > ${ALIAS}_se.ec.keep.fq

ls *.ec.keep.fq > ecfiles.txt
specialk ecfiles.txt -t $NSLOTS -l 31 -k 121 -s 2 -o ${ALIAS}
echo "declare -A COVS" >> coverage.txt
echo "declare -A CUTOFFS" > cutoffs.txt

#add one by one to the coverages files
for kmer in $(seq 31 2 121)
do
  MY_COV=$(Rscript get-peak.r ${ALIAS}-k${kmer}.histo)
  MY_CUTOFF=$(Rscript get-cutoff.r ${ALIAS}-k${kmer}.histo)
  echo "COVS[$kmer]=$MY_COV" >> coverage.txt
  echo "CUTOFFS[$kmer]=$MY_CUTOFF" >> cutoffs.txt
done


In [ ]:
for kind in mp1 mp2
do
  seqtk seq -r ${ALIAS}_${kind}.ec.keep.fq > ${ALIAS}_${kind}.ec.keep.rc.fq
done

In [ ]:
velveth velvet_seqs 33 -noHash -create_binary -fastq.gz \
-shortPaired $CLEANDIR/${ALIAS}_pe.ec.keep.fq.gz \
  -shortPaired2 $CLEANDIR/${ALIAS}_mp1.ec.keep.rc.fq.gz \
  -shortPaired3 $CLEANDIR/${ALIAS}_mp2.ec.keep.rc.fq.gz \
  -short4 $CLEANDIR/${ALIAS}_se.ec.keep.fq.gz

In [ ]:
for KMER in $(seq 31 2 121)
do
    mkdir velvet_$KMER
	cd velvet_$KMER
	ln -s ../velvet_seqs/CnyUnifiedSeq
	ln -s ../velvet_seqs/CnyUnifiedSeq.names
	cd ..

	#hash the reads
	OMP_NUM_THREADS=$NSLOTS velveth velvet_$KMER $KMER -reuse_binary
	#assemble the reads
	MY_COV=$COV
	OMP_NUM_THREADS=$NSLOTS velvetg velvet_$KMER -scaffolding no \
      -exp_cov $MY_COV -cov_cutoff ${CUTOFFS[$KMER]} -read_trkg yes \
      -ins_length 500 -ins_length2 3200 -ins_length3 8000 -min_contig_lgth 1000
done

In [ ]:
BEST_VELVET_CONTIGS=$(assembly_useful_stats.py --no_header velvet_*/contigs.fa|sort -nrk8|head -n1|cut -f 10)

In [ ]:
spades.py -t $NSLOTS --only-assembler -o spades_final_output -k 21,33,55,77,99,127 \
  --pe1-12 $CLEANDIR/${ALIAS}_pe.ec.keep.fq.gz --pe1-s $CLEANDIR/${ALIAS}_se.ec.keep.fq.gz \
  --hqmp1-12 $CLEANDIR/${ALIAS}_mp1.ec.keep.fq.gz --hqmp2-12 $CLEANDIR/${ALIAS}_mp2.ec.keep.fq.gz \
  --untrusted-contigs $BEST_VELVET_CONTIGS 